# Postverdad — Dashboard base (polarity & subjectivity)

Este notebook carga datos desde Postgres, usa las vistas propuestas y genera tablas y gráficos básicos.

**Requisitos previos**
- Variables de entorno: `POSTGRES_HOST`, `POSTGRES_PORT`, `POSTGRES_DB`, `POSTGRES_USER`, `POSTGRES_PASSWORD`.
- Vistas creadas ejecutando el archivo SQL: `postverdad_views.sql`.

> Nota: Si aún no creaste las vistas, puedes reemplazar las consultas a `vw_*` por los SELECT equivalentes en este notebook.


In [ ]:
import os
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
import matplotlib.pyplot as plt

# No fijar estilos ni colores específicos (requisito)
pd.set_option('display.max_rows', 20)

PG = {
    "host": os.getenv("POSTGRES_HOST", "localhost"),
    "port": int(os.getenv("POSTGRES_PORT", "5432")),
    "dbname": os.getenv("POSTGRES_DB", "postverdad"),
    "user": os.getenv("POSTGRES_USER", "postverdad"),
    "password": os.getenv("POSTGRES_PASSWORD", "postverdad"),
}
PG


In [ ]:
def read_sql_df(query, params=None):
    conn = psycopg2.connect(**PG)
    try:
        with conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(query, params or {})
            rows = cur.fetchall()
        return pd.DataFrame(rows)
    finally:
        conn.close()

## Cargar vistas

In [ ]:
df_articles = read_sql_df("SELECT * FROM vw_articles_labeled LIMIT 5000;")
df_monthly = read_sql_df("SELECT * FROM vw_monthly_media_metrics ORDER BY month DESC, source;")
df_matrix = read_sql_df("SELECT * FROM vw_last30_polarity_subjectivity_matrix;")

display(df_articles.head(10))
display(df_monthly.head(10))
display(df_matrix.head(10))


## Gráficos

In [ ]:
# Distribución polarity por fuente (top 10 por volumen en df_articles)
top_sources = (
    df_articles['source'].value_counts()
      .head(10)
      .index
)
sub = df_articles[df_articles['source'].isin(top_sources)]
plot_data = (
    sub.groupby(['source','polarity_label'])
       .size()
       .unstack(fill_value=0)
       .reindex(top_sources)
)

ax = plot_data.plot(kind='bar', figsize=(10,5))
ax.set_title('Distribución de polarity por fuente (top 10)')
ax.set_xlabel('Fuente')
ax.set_ylabel('Número de artículos')
plt.tight_layout()
plt.show()

In [ ]:
# Histograma de subjetividad
ax2 = df_articles['subjectivity'].dropna().plot(kind='hist', bins=30, figsize=(8,4))
ax2.set_title('Histograma de subjetividad')
ax2.set_xlabel('Subjectivity')
plt.tight_layout()
plt.show()

In [ ]:
# Evolución mensual por fuente (ejemplo con una fuente)
if not df_monthly.empty:
    fuente = df_monthly['source'].iloc[0]
    dm = df_monthly[df_monthly['source'] == fuente].sort_values('month')
    ax3 = dm.set_index('month')[['pct_neg','pct_neu','pct_pos']].plot(figsize=(10,5))
    ax3.set_title(f'Evolución % polarity — {fuente}')
    ax3.set_xlabel('Mes')
    ax3.set_ylabel('% artículos')
    plt.tight_layout()
    plt.show()